In [ ]:
!pip install torch torchvision torchaudio
!pip install diffusers
!pip install gradio
!pip install pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
import gradio as gr
from PIL import Image

In [ ]:

# Load Stable Diffusion model
authorization_token = ""
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

In [ ]:
# Initialize the pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch.float16, use_auth_token=authorization_token
)
pipe.to(device)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

unet/diffusion_pytorch_model.bin:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Keyword arguments {'use_auth_token': ''} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.33.1",
  "_name_or_path": "CompVis/stable-diffusion-v1-4",
  "feature_extractor": [
    "transformers",
    "CLIPFeatureExtractor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
# Function to generate the image
def generate_image(prompt):
    with autocast("cuda"):
        image = pipe(prompt, guidance_scale=8.5).images[0]
    return image


In [ ]:

# Create the Userinterface
def gradio_interface():
    with gr.Blocks(css="""
        .gradio-container {
            background: linear-gradient(to right, #4B0082, #8A2BE2); /* Gradient from Indigo to BlueViolet */
            color: white;
            font-family: 'Arial', sans-serif;
            height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
            text-align: center;
        }
        .gr-button {
            padding: 14px 35px;
            font-size: 18px;
            font-weight: bold;
            border-radius: 12px;
            color: white;
            border: none;
            cursor: pointer;
            margin: 10px;
            transition: background-color 0.3s ease;
        }
        .button-start {
            background-color: #FF6347; /* Tomato Red */
        }
        .button-start:hover {
            background-color: #FF4500; /* OrangeRed */
        }
        .button-generate {
            background-color: #00BFFF; /* Deep Sky Blue */
        }
        .button-generate:hover {
            background-color: #1E90FF; /* Dodger Blue */
        }
        .button-return {
            background-color: #9932CC; /* Dark Orchid */
        }
        .button-return:hover {
            background-color: #8A2BE2; /* BlueViolet */
        }
        .gr-label {
            font-size: 24px;
            font-weight: bold;
            color: white;
            text-align: center;
            margin-bottom: 20px;
        }
        .gr-textbox {
            background-color: rgba(255, 255, 255, 0.85);
            border-radius: 12px;
            padding: 12px;
            font-size: 18px;
            width: 100%;
            margin: 10px 0;
        }
        .gr-textbox input {
            background-color: rgba(255, 255, 255, 0.85);
            color: #333;
        }
        .gr-row {
            justify-content: center;
            align-items: center;
            margin-top: 20px;
        }
        .gr-markdown {
            text-align: center;
            margin-bottom: 30px;
        }
    """) as demo:

        gr.Markdown(
            """
            # Text-to-Image Using Stable Diffusion
            **Welcome to the image generation app!**
            Click **Start** to begin.
            """
        )

        # Layout elements
        with gr.Row():
            welcome_message = gr.Label(
                "Welcome to the generation app! Click 'Start' to enter your text prompt.",
                label=None,
            )
            output_image = gr.Image(label="Generated Image", visible=False)

        with gr.Row():
            start_button = gr.Button("Start", elem_classes=["gr-button", "button-start"])
            prompt_input = gr.Textbox(
                placeholder="Enter your prompt here...", visible=False, label="Enter your text prompt"
            )
            submit_button = gr.Button("Generate Image", visible=False, elem_classes=["gr-button", "button-generate"])
            next_button = gr.Button("Return & Enter New Prompt", visible=False, elem_classes=["gr-button", "button-return"])

        # Button interactions
        def start():
            return {
                welcome_message: "Please enter your text prompt below and click 'Generate Image'.",
                prompt_input: gr.update(visible=True),
                submit_button: gr.update(visible=True),
            }

        def submit(prompt):
            image = generate_image(prompt)
            return {
                output_image: gr.update(value=image, visible=True),
                next_button: gr.update(visible=True),
            }

        def return_to_start():
            return {
                prompt_input: gr.update(value="", visible=True),
                output_image: gr.update(visible=False),
                submit_button: gr.update(visible=True),
                next_button: gr.update(visible=False),
            }

        # Connect buttons to functions
        start_button.click(start, outputs=[welcome_message, prompt_input, submit_button])
        submit_button.click(submit, inputs=[prompt_input], outputs=[output_image, next_button])
        next_button.click(return_to_start, outputs=[prompt_input, output_image, submit_button, next_button])

    return demo

# Launch the Gradio app
demo = gradio_interface()
demo.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://72b4de73efbb4e604a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]